### Test the raster image dataloader

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader 
import numpy as np
import raster_relight as rr
import raster_dataloader as rd
import open3d as o3d
from matplotlib import pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
ds = rd.RasterDataset()

loading combined from /home/dtetruash/Thesis/datasets/nerf-blender/nerf_synthetic/intrinsic_tester_sphere/r_000_combined.png
loading normal from /home/dtetruash/Thesis/datasets/nerf-blender/nerf_synthetic/intrinsic_tester_sphere/r_000_normal.png
loading albedo from /home/dtetruash/Thesis/datasets/nerf-blender/nerf_synthetic/intrinsic_tester_sphere/r_000_albedo.png
loading depth from /home/dtetruash/Thesis/datasets/nerf-blender/nerf_synthetic/intrinsic_tester_sphere/r_000_depth.png
Bit depth used for depth remapping was 8
Occumpancy mask of shape (800, 800), and with 263345 occupied pixels.
Normal pixels range from 0 to 65535
Normals pixels shape: (263345, 3)
world_normals_shape after posing: (3, 263345)
world_normals shape after reshaping to camera_normals.shape: (263345, 3)
Posed_points shape in porject func: (640000, 3)
posed_points returned by rr has shape (640000, 3)
Loaded /home/dtetruash/Thesis/datasets/nerf-blender/nerf_synthetic/intrinsic_tester_sphere/lights_intrinsic_tester_s

In [4]:
dataset_list = list(ds)
len(dataset_list)

263345

In [5]:
normal = [sample[0] for sample in dataset_list]
albedo = [sample[1] for sample in dataset_list]
raster = [sample[2] for sample in dataset_list]


In [6]:
%%script echo skipping
albedo_img = np.ones((800,800,3)) * 255
albedo_img[ds._occupancy_mask] = np.array(albedo)
plt.imshow(albedo_img.astype(np.uint8))

skipping


In [7]:
%%script echo skipping
raster_img = np.ones((800,800,3)) * 255
raster_img[ds._occupancy_mask] = np.array(raster)
plt.imshow(raster_img.astype(np.uint8))

skipping


In [8]:
%%script echo skipping
normal_img = np.ones((800,800,3)) 
normal_img[ds._occupancy_mask] = (np.array(normal) * 0.5 + 0.5) 
plt.imshow(normal_img)

skipping


### Make sure the normals and light vectors are well aligned

In [17]:
pcd = ds._pcd

In [125]:
downpcd = pcd.voxel_down_sample(voxel_size=0.2)

In [126]:
o3d.visualization.draw_geometries([downpcd])

In [127]:
# make line set to the light point.

In [128]:
downpcd_points = np.asarray(downpcd.points)
len(downpcd_points)

390

In [129]:
pcd_points = np.asarray(ds._pcd.points)
pcd_points.shape

(263345, 3)

In [168]:
step = 26

In [173]:
ls_points = np.array([(i ,i + l*n) for (i,l,n) in zip(pcd_points, ds._light_vecs, ds._light_vecs_norms)][::step]).reshape(-1,3)
ls_points.shape

(20258, 3)

In [193]:
ls_points[1::2]

array([[0., 0., 2.],
       [0., 0., 2.],
       [0., 0., 2.],
       ...,
       [0., 0., 2.],
       [0., 0., 2.],
       [0., 0., 2.]])

In [174]:
ls_edges = [(i, i+1) for i in range(0,len(ls_points),2) ][::step]
len(ls_edges)

390

In [175]:

line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(ls_points),
    lines=o3d.utility.Vector2iVector(ls_edges),
)
line_set.colors = o3d.utility.Vector3dVector([(1,0,0)]*len(ls_edges))

In [182]:
mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=50)
mesh_sphere.compute_vertex_normals()
mesh_sphere.paint_uniform_color([0.1, 0.1, 0.7])

TriangleMesh with 4902 points and 9800 triangles.

In [198]:
# light point 
light_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.05)
light_sphere.compute_vertex_normals()
light_sphere.paint_uniform_color([0.5,0.5,0])
light_sphere.translate([0,0,2])

TriangleMesh with 762 points and 1520 triangles.

In [222]:
axis_set =  o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(np.array([[v*(a:=10),-v*a] for v in np.eye(3)]).reshape(-1,3)),
    lines=o3d.utility.Vector2iVector([(i, i+1) for i in range(0,6,2) ]),
)
axis_set.colors = o3d.utility.Vector3dVector(np.eye(3))

In [223]:
o3d.visualization.draw_geometries([downpcd,mesh_sphere, line_set, light_sphere, axis_set])

In [86]:
list(range(8))[::4]

[0, 4]

In [212]:
# make sphere point clou[d
basis = np.array([[0]*3]*3)
for i,b in enumerate(basis):
    b[i] +=1
basis

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

array([[ 10.,   0.,   0.],
       [-10.,  -0.,  -0.],
       [  0.,  10.,   0.],
       [ -0., -10.,  -0.],
       [  0.,   0.,  10.],
       [ -0.,  -0., -10.]])